# Day 6
## Part 1


In [2]:
from dataclasses import dataclass

@dataclass
class Point:
    x: int
    y: int

    def __add__(self, other):
        return self.__class__(self.x + other.x, self.y + other.y)

    def __sub__(self, other):
        return self.__class__(self.x - other.x, self.y - other.y)

    def __neg__(self):
        return self.__class__(-self.x, -self.y)

    def __hash__(self):
        return hash((self.x, self.y))

    def __lt__(self, other):
        if self.x < other.x:
            return True
        elif self.x > other.x:
            return False
        else:
            return self.y < other.y

    def __iter__(self):
        yield self.x
        yield self.y

    def __mod__(self, other):
        if isinstance(other, Point):
            return self.__class__(self.x % other.x, self.y % other.y)
        else:
            return self.__class__(self.x % other, self.y % other)
        
    def __mul__(self, multiple):
        return self.__class__(self.x * multiple, self.y * multiple)
    

N = Point(0, 1)
S = Point(0, -1)
W = Point(-1, 0)
E = Point(1, 0)

TURN_RIGHT = { N: E, E: S, S: W, W: N }

def parse_data(s):
    lab = {}
    for y, line in enumerate(reversed(s.strip().splitlines())):
        for x, c in enumerate(line):
            if c == "^":
                starting_point = Point(x, y)
            lab[Point(x, y)] = c
    return starting_point, lab

def part_1(data):
    loc, lab = data
    visited = {loc}
    d = N
    while loc + d in lab:
        if lab[loc + d] == "#":
            d = TURN_RIGHT[d]
        else:
            loc = loc + d
            visited.add(loc)
    return len(visited)

test_data = parse_data("""....#.....
.........#
..........
..#.......
.......#..
..........
.#..^.....
........#.
#.........
......#...""")

assert part_1(test_data) == 41

In [4]:
data = parse_data(open("input").read())
part_1(data)

4988

## Part 2

Brute force for now.

In [20]:
import tqdm

def gets_in_loop(starting_position, lab, new_obstacle):
    loc = starting_position
    d = N
    visited = {(loc, d)}
    while loc + d in lab:
        if lab[loc + d] == "#" or loc + d == new_obstacle:
            d = TURN_RIGHT[d]
        else:
            loc = loc + d
            if (loc, d) in visited:
                return True
            visited.add((loc, d))
    return False

def part_2(data):
    starting_position, lab = data
    result = 0

    for new_obstacle in tqdm.tqdm([x for x in lab if lab[x] == "."]):
        if gets_in_loop(starting_position, lab, new_obstacle):
            result += 1
    return result

assert part_2(test_data) == 6

100%|███████████████████████████████████████████████████| 91/91 [00:00<00:00, 12282.20it/s]


In [21]:
part_2(data)

100%|████████████████████████████████████████████████| 16081/16081 [02:55<00:00, 91.88it/s]


1697

That's slow. We don't have to check every location in the lab for an extra obstacle, just those that would have blocked the original route.

In [27]:
def possible_obstacles(data):
    loc, lab = data
    d = N
    visited = {loc}
    while loc + d in lab:
        if lab[loc + d] == "#":
            d = TURN_RIGHT[d]
        else:
            loc = loc + d
            if loc not in visited:
                yield loc
                visited.add(loc)

def part_2_v2(data):
    starting_position, lab = data
    result = 0

    for new_obstacle in tqdm.tqdm(list(possible_obstacles(data))):
        if gets_in_loop(starting_position, lab, new_obstacle):
            result += 1
    return result

assert part_2_v2(test_data) == 6

100%|████████████████████████████████████████████████████| 40/40 [00:00<00:00, 6358.38it/s]


In [28]:
part_2_v2(data)

100%|█████████████████████████████████████████████████| 4987/4987 [00:36<00:00, 135.43it/s]


1697

Still slow though.